# Music Predictor
Using this file to predict the genre of a given music file

In [1]:
from music_dealer import MusicDealer
from models import CnnModel, CrnnLongModel, CrnnModel, MambaVisionModel, DinoModel
import json
from Paras import Para
import os

2024-08-09 10:02:14.828816: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-09 10:02:15.362208: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-12.2/lib64
2024-08-09 10:02:15.362259: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:/usr/local/cuda-12.2/lib64
2024-08-09 10:02:15.36

In [2]:
WEIGHT_PATH = "./"
cnn_dealer = MusicDealer(WEIGHT_PATH + "modelsCnnModel.pt", CnnModel())
MambaVision_dealer = MusicDealer(WEIGHT_PATH + "modelsMambaVisionModel.pt", MambaVisionModel().cpu())
Dino_dealer = MusicDealer(WEIGHT_PATH + "modelsDinoModel.pt", DinoModel(), 256)

Some weights of ViTModel were not initialized from the model checkpoint at facebook/dino-vits8 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Sample Music  Paths

In [36]:
data_path = '../sample_music'
genre_path_dict = dict()
label_dict = Para.r_dictionary

for (dirpath, dirnames, filenames) in os.walk(data_path):
    if dirnames:
        for idx, genre in enumerate(dirnames):
            genre_path_dict[genre] = list()
        continue
    else:
        genre = dirpath.split('/')[-1]
        for au_file in filenames:
            if 'mp3' in au_file:
                current_path = dirpath + '/' + au_file
                genre_path_dict[genre].append(current_path)

total_file = sum([len(genre_path_dict[genre]) for genre in genre_path_dict])
total_file, label_dict, genre_path_dict

(1,
 {'pop': 0,
  'metal': 1,
  'disco': 2,
  'blues': 3,
  'reggae': 4,
  'classical': 5,
  'rock': 6,
  'hiphop': 7,
  'country': 8,
  'jazz': 9},
 {'reggae': ['../sample_music/reggae/Punch-Long-Version.mp3']})

In [37]:
def test_model(dealer, save_name):
    test_result = dict()
    for key in genre_path_dict:
        test_result[key] = 0
        print('test on sample {0}'.format(key))
        for file_name in genre_path_dict.get(key):
            res1, res2, res3, score = dealer.get_genre(file_name)
            if res1 == label_dict.get(key):
                test_result[key] += score.get(res1)
            elif res2 == label_dict.get(key):
                test_result[key] += score.get(res2)
            elif res3 == label_dict.get(key):
                test_result[key] += score.get(res3)
            print("*"*99)
        test_result[key] /= 100
        print(test_result[key])
        print("+"*99)
    with open(Para.LOG_SAVE_FOLD + save_name, 'w+') as f:
        json.dump(test_result, f)
    return

# Test on different models


In [38]:
test_model(cnn_dealer, 'cnnTest.json')

test on sample reggae
Test on Punch-Long-Version.mp3


Genre reggae: 100.0%
***************************************************************************************************
1.0
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [39]:
test_model(MambaVision_dealer, 'MambaVisionTest.json')

test on sample reggae
Test on Punch-Long-Version.mp3
Genre reggae: 84.87%
Genre hiphop: 15.13%
***************************************************************************************************
0.8487127773237586
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [40]:
test_model(Dino_dealer, 'DinoTest.json')

test on sample reggae
Test on Punch-Long-Version.mp3
Genre hiphop: 86.72%
Genre disco: 13.28%
***************************************************************************************************
0.0
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
